## Assignment 2 Python Solution Template

In [3]:
# some imports you may require
import simpy
import random
import numpy as np
import pandas as pd
import time
import math

## Part A

In [15]:
### write your solution here
class PartA: # regular check out

    def __init__(self, arrival_mean, service_mean, service_sd, num_customers):
        self.env = simpy.Environment()
        self.counter = simpy.Resource(self.env, capacity=1)
        self.arrival_mean = arrival_mean
        self.service_mean = service_mean
        self.service_sd = service_sd
        self.num_customers = num_customers

        # variables to keep track
        # TODO: insert code here for any additional variables you would like to keep track of
        self.customers = []
        self.queue = [0]
        self.timing = []


    def customer_arrival(self, cust_id):
        # TODO: implement the customer arrival process to be used in arrival_process
        self.customers[cust_id]["arrival_time"] = self.env.now
        with self.counter.request() as request:

            # wait until server is available
            yield request

            # record the time the customer is served
            self.customers[cust_id]["serve_time"] = self.env.now
            
            # keep track of queue length
            self.queue.append(self.queue[-1]-1) 
            self.timing.append(self.env.now)
            
            # service time
            service_time = max(random.normalvariate(self.service_mean, self.service_sd), 0.5)
            yield self.env.timeout(service_time)

            self.customers[cust_id]["leave_time"] = self.env.now


    def arrival_process(self):
        for cust_id in range(self.num_customers):
            # TODO: use the customer_arrival method to implement the arrival process for each customer
            yield self.env.timeout(random.expovariate(self.arrival_mean)) 

            # create a dictionary to store the data of each customer
            cust_data = {"arrival_time":self.env.now, "serve_time":0, "leave_time":0}
            self.customers.append(cust_data)

            # start the customer arrival process
            self.env.process(self.customer_arrival(cust_id))

            # keep track of queue length
            self.queue.append(self.queue[-1]+1)
            self.timing.append(self.env.now)
            

    def get_avg_sys_time(self):
        # TODO: returns the average system time
        sys_time = [data["leave_time"] - data["arrival_time"] for data in self.customers if data["leave_time"] > 0]
        return np.mean(sys_time)
        

    def get_avg_wait_time(self):
        # TODO: returns the average wait time
        wait_time = [data["serve_time"]-data["arrival_time"] for data in self.customers if data["leave_time"]>0]
        return np.mean(wait_time)
        

    def get_avg_queue_length(self):
        # TODO: returns the average queue length
        k = min(len(self.queue), len(self.timing)) 
        total_queue_time, total_time = 0 , 0
        for i in range(1, k):
            time_diff = self.timing[i] - self.timing[i-1]
            total_queue_time += self.queue[i] * time_diff
            total_time += time_diff
        return total_queue_time / total_time
        

    def run(self):
        self.env.process(self.arrival_process())
        self.env.run()
        return self.get_avg_wait_time(), self.get_avg_sys_time(), self.get_avg_queue_length()

In [16]:
def run_experiment(num_simulations=100):

    # DO NOT MODIFY THIS
    avg_wq_list = []
    avg_w_list = []
    avg_lq_list = []

    for _ in range(num_simulations):
        # TODO: run the experiment using PartA and append the results (wl, w, and lq) to avg_wq_list, avg_w_list and avg_lq_list respectively
        
        parta = PartA( # fill this part yourself
            arrival_mean=1, 
            service_mean=3, 
            service_sd=2, 
            num_customers=50
        )
        parta.run()
        wq, w, lq = parta.run()
        avg_wq_list.append(wq)
        avg_w_list.append(w)
        avg_lq_list.append(lq)

    # DO NOT MODIFY THIS
    print("avg_wq: ", np.mean(np.array(avg_wq_list)))
    print("avg_w:  ", np.mean(np.array(avg_w_list)))
    print("avg_lq: ", np.mean(np.array(avg_lq_list)))


    return np.array([
        np.mean(np.array(avg_wq_list)), np.mean(np.array(avg_w_list)), np.mean(np.array(avg_lq_list))
    ])

In [17]:
random.seed(42) # random seed to be used
results = run_experiment()

avg_wq:  51.8914349817696
avg_w:   54.983944457749914
avg_lq:  16.83927524116473


## Part B


In [7]:
class PartB: # self check out
    
    def __init__(self, arrival_mean, service_a, service_b, num_servers, num_arrivals, total_sim_time):        
        self.env = simpy.Environment()
        self.servers = simpy.Resource(self.env, capacity = num_servers)
        self.arrival_mean = arrival_mean
        self.service_rate = (service_a, service_b)
        self.num_arrivals = num_arrivals
        self.total_sim_time = total_sim_time

        # variables to keep track
        # TODO: insert code here for any additional variables you would like to keep track of
        self.customers = []
        self.queue = [0]
        self.timing = []
        

    def generate_customer_process(self, cust_id):
        # TODO: implement the customer arrival process to be used in generate_arrival_process
        self.customers[cust_id]["arrival_time"] = self.env.now
        with self.servers.request() as request:
            # wait until server is available
            yield request

            # record the time the customer is served
            self.customers[cust_id]["serve_time"] = self.env.now
            
            # keep track of queue length
            self.queue.append(self.queue[-1]-1) 
            self.timing.append(self.env.now)
            
            # service time
            service_time = random.uniform(self.service_rate[0], self.service_rate[1])
            yield self.env.timeout(service_time) 

            self.customers[cust_id]["leave_time"] = self.env.now        

        
    def generate_arrival_process(self, num_arrivals):
        for cust_id in range(num_arrivals):
            # TODO: use the generate_customer_process method to implement the arrival process for each customer
            yield self.env.timeout(random.expovariate(1/self.arrival_mean)) 

            # create a dictionary to store the data of each customer
            cust_data = {"arrival_time":self.env.now, "serve_time":0, "leave_time":0}
            self.customers.append(cust_data)

            # start the customer arrival process
            self.env.process(self.generate_customer_process(cust_id))

            # keep track of queue length
            self.queue.append(self.queue[-1]+1)
            self.timing.append(self.env.now)          
            
    
    
    def get_avg_sys_time(self):
        # TODO: returns the average system time
        sys_time = [cust["leave_time"] - cust["arrival_time"] for cust in self.customers if cust["leave_time"] > 0]
        return np.mean(sys_time)       
        

    
    def get_avg_wait_time(self):
        # TODO: returns the average wait time
        wait_time = [data["serve_time"]-data["arrival_time"] for data in self.customers if data["leave_time"]>0]
        return np.mean(wait_time)      
        
    
    
    def get_avg_queue_length(self):
        # TODO: returns the average queue length
        k = min(len(self.queue), len(self.timing)) 
        total_queue_time, total_time = 0 , 0
        for i in range(1, k):
            time_diff = self.timing[i] - self.timing[i-1]
            total_queue_time += self.queue[i] * time_diff
            total_time += time_diff
        return total_queue_time / total_time        
        

    def run(self):
        self.env.process(self.generate_arrival_process(self.num_arrivals))
        self.env.run(until = self.total_sim_time)
        return self.get_avg_wait_time(), self.get_avg_sys_time(), self.get_avg_queue_length()
        


In [8]:
def run_experiment(# fill in the below values yourself
                    max_num_items, 
                    arrival_mean=1, 
                    num_self_service=3, 
                    num_cashier=2,
                    total_sim_time=math.inf,
                    num_simulations=100
                ):

    # DO NOT MODIFY THIS
    avg_wait_time_list_self_service = []
    avg_sys_time_list_self_service = []
    avg_queue_length_list_self_service = []

    avg_wait_time_list_cashier = []
    avg_sys_time_list_cashier = []
    avg_queue_length_list_cashier = []

    parameters = { # a lookup dictionary provided for students stating the b parameter and num_arrivals to the self-service kiosk. feel free to use them
        2: {"service_b": 3.2, "num_arrivals": 60},
        3: {"service_b": 3.5, "num_arrivals": 70},
        4: {"service_b": 3.8, "num_arrivals": 80},
        5: {"service_b": 4.1, "num_arrivals": 100},
        6: {"service_b": 4.4, "num_arrivals": 120},
        7: {"service_b": 4.7, "num_arrivals": 140},
        8: {"service_b": 5.0, "num_arrivals": 160},
    }

    for _ in range(num_simulations):
        partb = PartB( # fill in the below values yourself
                        arrival_mean=1, 
                        service_a=3,
                        service_b=parameters[max_num_items]["service_b"], 
                        num_servers=num_self_service,
                        num_arrivals=parameters[max_num_items]["num_arrivals"], 
                        total_sim_time=math.inf
                    )
        wq, w, lq = partb.run()
        avg_wait_time_list_self_service.append(wq)
        avg_sys_time_list_self_service.append(w)
        avg_queue_length_list_self_service.append(lq)

        parta = PartA( # fill in the below values yourself
            arrival_mean=1, 
            service_mean=3, 
            service_sd=2, 
            num_customers=int((200-parameters[max_num_items]["num_arrivals"])/num_cashier)
        )
        wq, w, lq = parta.run()
        avg_wait_time_list_cashier.append(wq)
        avg_sys_time_list_cashier.append(w)
        avg_queue_length_list_cashier.append(lq)

    print("avg_wait_time_list_self_service:", np.mean(np.array(avg_wait_time_list_self_service)))
    print("avg_sys_time_list_self_service:", np.mean(np.array(avg_sys_time_list_self_service)))
    print("avg_queue_length_list_self_service:", np.mean(np.array(avg_queue_length_list_self_service)))
    print("avg_wait_time_list_cashier", np.mean(np.array(avg_wait_time_list_cashier)))
    print("avg_sys_time_list_cashier", np.mean(np.array(avg_sys_time_list_cashier)))
    print("avg_queue_length_list_cashier", np.mean(np.array(avg_queue_length_list_cashier)))


    return np.array([
        max_num_items, 
        np.mean(np.array(avg_wait_time_list_self_service)), 
        np.mean(np.array(avg_sys_time_list_self_service)), 
        np.mean(np.array(avg_queue_length_list_self_service)), 
        np.mean(np.array(avg_wait_time_list_cashier)), 
        np.mean(np.array(avg_sys_time_list_cashier)), 
        np.mean(np.array(avg_queue_length_list_cashier))
    ])

In [110]:
all_results = []
for i in range(2, 9):
    print("=======================================")
    print("Maximum Basket Size:", i)
    random.seed(42) # random seed used in lab 5
    results = run_experiment( # fill in any other parameters as needed
        max_num_items=i)
    all_results.append(results)
    print("=======================================")
    # break

all_results = np.array(all_results)

Maximum Basket Size: 2
avg_wait_time_list_self_service: 3.824264277753606
avg_sys_time_list_self_service: 6.923834093625309
avg_queue_length_list_self_service: 3.6463035917359763
avg_wait_time_list_cashier 71.90329398090685
avg_sys_time_list_cashier 74.9964992023061
avg_queue_length_list_cashier 23.484325686255424
Maximum Basket Size: 3
avg_wait_time_list_self_service: 5.1621821827919145
avg_sys_time_list_self_service: 8.412550500663192
avg_queue_length_list_self_service: 4.710062131075128
avg_wait_time_list_cashier 68.37111699621349
avg_sys_time_list_cashier 71.48253566027046
avg_queue_length_list_cashier 22.227660374023785
Maximum Basket Size: 4
avg_wait_time_list_self_service: 7.459793584021391
avg_sys_time_list_self_service: 10.86091182902303
avg_queue_length_list_self_service: 6.556176159551236
avg_wait_time_list_cashier 62.79408545966544
avg_sys_time_list_cashier 65.91685234336106
avg_queue_length_list_cashier 20.349309467735218
Maximum Basket Size: 5
avg_wait_time_list_self_serv

In [111]:
# convert to df
results_df = pd.DataFrame(all_results, columns=["Max Number of Items", "Average Wait Time", "Average System", "Average Queue", "Average Wait Time", "Average System", "Average Queue"])
results_df.head(10)

,Max Number of Items,Average Wait Time,Average System,Average Queue,Average Wait Time,Average System,Average Queue
0,2.0,3.824264,6.923834,3.646304,71.903294,74.996499,23.484326
1,3.0,5.162182,8.412551,4.710062,68.371117,71.482536,22.227660
2,4.0,7.459794,10.860912,6.556176,62.794085,65.916852,20.349309
3,5.0,10.839461,14.389382,9.187484,50.817175,53.924341,16.623460
4,6.0,15.705295,19.408024,12.723089,41.537852,44.656261,13.582818
5,7.0,20.386859,24.238368,15.950859,31.053061,34.178729,10.212309
6,8.0,27.282387,31.280511,20.547115,20.010841,23.133255,6.682693


In [112]:
results_df3 = results_df.copy()
results_df3['Total Number of Customers Queueing'] = results_df3.iloc[:, 3] + results_df3.iloc[:, 6] * 2  
results_df3.head(10)

,Max Number of Items,Average Wait Time,Average System,Average Queue,Average Wait Time,Average System,Average Queue,Total Number of Customers Queueing
0,2.0,3.824264,6.923834,3.646304,71.903294,74.996499,23.484326,50.614955
1,3.0,5.162182,8.412551,4.710062,68.371117,71.482536,22.227660,49.165383
2,4.0,7.459794,10.860912,6.556176,62.794085,65.916852,20.349309,47.254795
3,5.0,10.839461,14.389382,9.187484,50.817175,53.924341,16.623460,42.434403
4,6.0,15.705295,19.408024,12.723089,41.537852,44.656261,13.582818,39.888725
5,7.0,20.386859,24.238368,15.950859,31.053061,34.178729,10.212309,36.375477
6,8.0,27.282387,31.280511,20.547115,20.010841,23.133255,6.682693,33.912502


## Part C

In [113]:
all_results = []
for i in range(2, 9):
    print("=======================================")
    print("Maximum Basket Size:", i)
    random.seed(42) # random seed used in lab 5
    results = run_experiment( # fill in any other parameters as needed
        max_num_items=i, num_self_service=2)
    all_results.append(results)
    print("=======================================")

all_results = np.array(all_results)

Maximum Basket Size: 2
avg_wait_time_list_self_service: 16.66557472124117
avg_sys_time_list_self_service: 19.765346351809402
avg_queue_length_list_self_service: 10.946801026928718
avg_wait_time_list_cashier 71.90329398090685
avg_sys_time_list_cashier 74.9964992023061
avg_queue_length_list_cashier 23.484325686255424
Maximum Basket Size: 3
avg_wait_time_list_self_service: 22.047530356692207
avg_sys_time_list_self_service: 25.299578195205335
avg_queue_length_list_self_service: 13.758673188490773
avg_wait_time_list_cashier 68.37111699621349
avg_sys_time_list_cashier 71.48253566027046
avg_queue_length_list_cashier 22.227660374023785
Maximum Basket Size: 4
avg_wait_time_list_self_service: 27.26210734020431
avg_sys_time_list_self_service: 30.66224959316409
avg_queue_length_list_self_service: 16.258564778510074
avg_wait_time_list_cashier 62.79408545966544
avg_sys_time_list_cashier 65.91685234336106
avg_queue_length_list_cashier 20.349309467735218
Maximum Basket Size: 5
avg_wait_time_list_self_

In [114]:
# convert to df
results_df2 = pd.DataFrame(all_results, columns=["Max Number of Items", "Average Wait Time", "Average System", "Average Queue", "Average Wait Time", "Average System", "Average Queue"])
results_df2.head(10)

,Max Number of Items,Average Wait Time,Average System,Average Queue,Average Wait Time,Average System,Average Queue
0,2.0,16.665575,19.765346,10.946801,71.903294,74.996499,23.484326
1,3.0,22.047530,25.299578,13.758673,68.371117,71.482536,22.227660
2,4.0,27.262107,30.662250,16.258565,62.794085,65.916852,20.349309
3,5.0,39.167571,42.723786,22.283955,50.817175,53.924341,16.623460
4,6.0,50.753312,54.452157,27.704163,41.537852,44.656261,13.582818
5,7.0,65.075687,68.929399,34.063002,31.053061,34.178729,10.212309
6,8.0,79.152032,83.148797,39.895164,20.010841,23.133255,6.682693


In [116]:
# in part c just change number of servers from part b

In [117]:
results_df4 = results_df2.copy()
results_df4['Total Number of Customers Queueing'] = results_df4.iloc[:, 3] + results_df4.iloc[:, 6] * 2  
results_df4.head(10)

,Max Number of Items,Average Wait Time,Average System,Average Queue,Average Wait Time,Average System,Average Queue,Total Number of Customers Queueing
0,2.0,16.665575,19.765346,10.946801,71.903294,74.996499,23.484326,57.915452
1,3.0,22.047530,25.299578,13.758673,68.371117,71.482536,22.227660,58.213994
2,4.0,27.262107,30.662250,16.258565,62.794085,65.916852,20.349309,56.957184
3,5.0,39.167571,42.723786,22.283955,50.817175,53.924341,16.623460,55.530874
4,6.0,50.753312,54.452157,27.704163,41.537852,44.656261,13.582818,54.869798
5,7.0,65.075687,68.929399,34.063002,31.053061,34.178729,10.212309,54.487619
6,8.0,79.152032,83.148797,39.895164,20.010841,23.133255,6.682693,53.260551
